In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime as dt 
from tqdm import tqdm 
import yfinance as yf

#import sqlite3 as sql 
# options_file_path = '../../data/options/options.db'
# price_file_path = '../../data/prices/stocks.db'

# option_db = sql.connect(options_file_path)
# price_db = sql.connect(price_file_path)


print(dt.datetime.now())

2024-04-25 16:17:01.789917


In [2]:
stock = 'intc'
tk = yf.Ticker(stock)
price = tk.history().iloc[-1]['Close']
exps = tk.options[:3]
options = []
for e in exps:
    opt = tk.option_chain(e)
    calls = opt.calls
    calls['type'] = 'Call'
    puts = opt.puts
    puts['type'] = 'Put'
    option_df = pd.concat([calls, puts])
    option_df['expiry'] = e
    option_df['stk_price'] = price  
    option_df.columns = [c.lower() for c in option_df.columns]
    options.append(option_df)    
    
odf = pd.concat(options)

In [3]:


# Max Call Strike for ITM options, Min Put Strike for ITM options
itm = odf.copy()
call_strike = itm[(itm['type'] == 'Call') & (itm['strike'] < itm.stk_price)]['strike'].max()
put_strike = itm[(itm['type'] == 'Put') & (itm['strike'] > itm.stk_price)]['strike'].min()
print(call_strike, put_strike)

cols = ['expiry','stk_price','type','strike','ask']
call_em = itm[(itm.strike == call_strike) & (itm.type == 'Call')][cols]
put_em = itm[(itm.strike == put_strike) & (itm.type == 'Put')][cols]
em = pd.concat([call_em, put_em]).groupby(['expiry']).agg({'stk_price':'first', 'ask':'sum'})
em['%'] = (0.85 * em['ask']) / em['stk_price']
em.rename(columns = {'ask':'em'}, inplace = True)
em

35.0 35.5


,stk_price,em,%
expiry,,,
2024-04-26,35.110001,2.96,0.071660
2024-05-03,35.110001,3.33,0.080618
2024-05-10,35.110001,1.68,0.040672


In [5]:
# Max Call Strike for ITM options, Min Put Strike for ITM options
itm = odf[odf['inthemoney'] == True]
call_strike = itm[itm['type'] == 'Call']['strike'].max()
put_strike = itm[itm['type'] == 'Put']['strike'].min()
print(call_strike, put_strike)

cols = ['expiry','stk_price','type','strike','ask', 'volume', 'openinterest', 'impliedvolatility']
em = itm[itm.strike.isin([call_strike, put_strike])][cols].groupby(['expiry']).agg({'stk_price':'first', 'ask':'sum'})
em['%'] = (0.85 * em['ask']) / em['stk_price']
em.rename(columns = {'ask':'em'}, inplace = True)
em

35.0 35.5


,stk_price,em,%
expiry,,,
2024-04-26,35.110001,2.96,0.071660
2024-05-03,35.110001,3.33,0.080618
2024-05-10,35.110001,1.68,0.040672


In [6]:
odf[cols]

,expiry,stk_price,type,strike,ask,volume,openinterest,impliedvolatility
0,2024-04-26,35.110001,Call,25.0,10.60,1.0,81,3.125002
1,2024-04-26,35.110001,Call,27.0,9.65,NaN,12,4.933598
2,2024-04-26,35.110001,Call,28.0,7.30,10.0,11,2.195317
3,2024-04-26,35.110001,Call,29.0,6.85,5.0,8,2.966799
4,2024-04-26,35.110001,Call,30.0,6.50,103.0,286,2.582035
...,...,...,...,...,...,...,...,...
19,2024-05-10,35.110001,Put,44.0,10.95,2.0,127,0.652347
20,2024-05-10,35.110001,Put,45.0,11.95,3.0,44,0.703128
21,2024-05-10,35.110001,Put,46.0,12.95,2.0,55,0.714847
22,2024-05-10,35.110001,Put,47.0,13.90,1.0,98,0.718753


In [7]:
mmm = em.em.iloc[0]
call_targets = [price + (i * mmm) for i in range(1, 3)]
put_targets = [price - (i * mmm) for i in range(1, 3)]

call_targets

[38.07000061035156, 41.030000610351564]

In [8]:
avail_strikes = odf.strike.unique()
# Get the nearest strike to x 
def nearest_strike(x, strikes):
    return min(strikes, key=lambda y:abs(y-x))

# For each expiration, get the unique strikes, then find the target 
targets = {}; flags = {}
for e in exps: 
    strikes = odf[odf.expiry == e].strike.unique()
    targets[e] = {'Call': [nearest_strike(x, strikes) for x in call_targets], 
                'Put': [nearest_strike(x, strikes) for x in put_targets]}
    flags[e] = {'Call': [1, 2], 'Put': [1, 2]}

targets

{'2024-04-26': {'Call': [38.0, 41.0], 'Put': [32.0, 29.0]},
 '2024-05-03': {'Call': [38.0, 41.0], 'Put': [32.0, 29.0]},
 '2024-05-10': {'Call': [38.0, 41.0], 'Put': [32.0, 29.0]}}

In [9]:
out = []
for i in targets:
    o = odf[odf.expiry == i][cols]
    for j in targets[i]:
        for k in targets[i][j]:
            tmp = o[(o.type == j) & (o.strike == k)].copy()
            tmp_o = flags[i][j][targets[i][j].index(k)]
            tmp.insert(3, 'sd', tmp_o)
            out.append(tmp)

In [10]:
out_df = pd.concat(out).reset_index(drop = True)

def break_even(option_type, strike, option_price, stock_price):
    if option_type == 'Call':
        out = ((strike + option_price) - stock_price) / stock_price
    if option_type == 'Put':
        out = ((strike - option_price) - stock_price) / stock_price
    return out

out_df['be'] = out_df.apply(lambda x: break_even(x['type'], x['strike'], x['ask'], x['stk_price']), axis = 1)
out_df = out_df.set_index(['expiry']).join(em['em'])
out_df

,stk_price,type,sd,strike,ask,volume,openinterest,impliedvolatility,be,em
expiry,,,,,,,,,,
2024-04-26,35.110001,Call,1,38.0,0.38,12494.0,7967,1.251957,0.093136,2.96
2024-04-26,35.110001,Call,2,41.0,0.09,861.0,3170,1.308597,0.170322,2.96
2024-04-26,35.110001,Put,1,32.0,0.26,7168.0,2688,1.246098,-0.095984,2.96
2024-04-26,35.110001,Put,2,29.0,0.03,1844.0,407,1.281254,-0.174879,2.96
2024-05-03,35.110001,Call,1,38.0,0.55,4185.0,3977,0.691409,0.097978,3.33
2024-05-03,35.110001,Call,2,41.0,0.16,442.0,2464,0.701175,0.172316,3.33
2024-05-03,35.110001,Put,1,32.0,0.39,1084.0,1330,0.683597,-0.099687,3.33
2024-05-03,35.110001,Put,2,29.0,0.07,91.0,974,0.703128,-0.176018,3.33
2024-05-10,35.110001,Call,1,38.0,0.61,512.0,1822,0.543950,0.099687,1.68


In [11]:
df = out_df.copy()
df.index = pd.to_datetime(df.index)
current_em = df['em'].iloc[0]
current_stock_price = df.stk_price.iloc[0]
current_expiration = df.index[0]
one_sd_call = df[(df.sd == 1) & (df['type'] == 'Call') & (df.index == current_expiration)].iloc[0]
one_sd_put = df[(df.sd == 1) & (df['type'] == 'Put') & (df.index == current_expiration)].iloc[0]
two_sd_call = df[(df.sd == 2) & (df['type'] == 'Call') & (df.index == current_expiration)].iloc[0]
two_sd_put = df[(df.sd == 2) & (df['type'] == 'Put') & (df.index == current_expiration)].iloc[0]
txt = f'''

${stock.upper()} Pricing In about {current_em:.2%} move by {current_expiration:%m/%d}
For calls, the ${one_sd_call.strike} strike is priced at ${one_sd_call.ask:.2f} needing a {one_sd_call.be:.2%} move to break even. 
The ${two_sd_call.strike} strike is priced at ${two_sd_call.ask:.2f} needing a {two_sd_call.be:.2%} move to break even.

For Puts the ${one_sd_put.strike} strike is priced at ${one_sd_put.ask:.2f} needing a {one_sd_put.be:.2%} move to break even.
The ${two_sd_put.strike} strike priced at ${two_sd_put.ask:.2f} needing a {two_sd_put.be:.2%} move to break even.
'''
print(txt)



$INTC Pricing In about 296.00% move by 04/26
For calls, the $38.0 strike is priced at $0.38 needing a 9.31% move to break even. 
The $41.0 strike is priced at $0.09 needing a 17.03% move to break even.

For Puts the $32.0 strike is priced at $0.26 needing a -9.60% move to break even.
The $29.0 strike priced at $0.03 needing a -17.49% move to break even.

